In [ ]:
# save as download_copernicus.py or paste into a notebook cell
import os
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from datetime import date, timedelta
import time
import sys

COPERNICUS_USER = 
COPERNICUS_PASSWORD = 

FT_WKT = "POLYGON((76.8943 28.3254,77.4758 28.3254,77.4758 28.6298,76.8943 28.6298,76.8943 28.3254))"

DATA_COLLECTION = "SENTINEL-1"
DAYS_BACK = 10 
def get_keycloak_token(username: str, password: str) -> str:

    if not username or not password:
        raise ValueError("Copernicus credentials missing. Set COPERNICUS_USER and COPERNICUS_PASSWORD env vars.")
    auth_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    r = requests.post(auth_url, data=data, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"Keycloak auth failed: {r.status_code} {r.text}")
    return r.json()["access_token"]

def build_odata_query(collection_name: str, ft_wkt: str, start_date: str, end_date: str, top: int = 100) -> str:

    ft_odata = f"POLYGON(({','.join([pt for pt in ft_wkt.replace('POLYGON((','').replace('))','').split(',')])}))"
    # Use raw ft_wkt in OData Intersects expression
    odata_filter = (
        f"Collection/Name eq '{collection_name}'"
        f" and ContentDate/Start ge {start_date}T00:00:00Z"
        f" and ContentDate/Start le {end_date}T23:59:59Z"
        f" and OData.CSC.Intersects(area=geography'SRID=4326;{ft_wkt}')"
    )
    q = (
        "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?"
        f"$filter={odata_filter}&$count=True&$top={top}"
    )
    return q

def try_download_product(session: requests.Session, headers: dict, product_id: str, outname: str, timeout: int = 300):
    
    candidates = []
    if "-" in product_id and any(c.isalpha() for c in product_id):
        candidates.append(f"https://zipper.dataspace.copernicus.eu/odata/v1/Products(guid'{product_id}')/$value")
    # plain
    candidates.append(f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value")
    # naive fallback (quoted)
    candidates.append(f"https://zipper.dataspace.copernicus.eu/odata/v1/Products('{product_id}')/$value")

    last_exc = None
    for url in candidates:
        try:
            r = session.get(url, headers=headers, stream=True, allow_redirects=True, timeout=timeout)
            
            if r.status_code == 200:
                with open(outname, "wb") as fh:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            fh.write(chunk)
                print(f"Downloaded: {outname}  (via {url})")
                return True
            if r.status_code in (301, 302, 303, 307) and "Location" in r.headers:
                signed_url = r.headers["Location"]
                print(f"Redirected to signed URL: {signed_url[:200]}...")
                rr = requests.get(signed_url, stream=True, timeout=timeout)
                if rr.status_code == 200:
                    with open(outname, "wb") as fh:
                        for chunk in rr.iter_content(chunk_size=8192):
                            if chunk:
                                fh.write(chunk)
                    print(f"Downloaded: {outname}  (via signed url)")
                    return True
                else:
                    last_exc = RuntimeError(f"signed URL returned {rr.status_code}: {rr.text[:200]}")
            else:
                last_exc = RuntimeError(f"download attempt returned {r.status_code}: {r.text[:300]}")
        except Exception as e:
            last_exc = e
            print(f"Attempt URL {url} failed with exception: {e}")

    raise RuntimeError(f"All download attempts failed for product {product_id}. Last error: {last_exc}")

def main():
    if not COPERNICUS_USER or not COPERNICUS_PASSWORD:
        print("ERROR: set COPERNICUS_USER and COPERNICUS_PASSWORD as environment variables (recommended).")
        return

    today = date.today()
    start_dt = (today - timedelta(days=DAYS_BACK)).strftime("%Y-%m-%d")
    end_dt = today.strftime("%Y-%m-%d")

    print(f"Query window: {start_dt} -> {end_dt}")

    # get token
    print("Requesting Keycloak token...")
    token = get_keycloak_token(COPERNICUS_USER, COPERNICUS_PASSWORD)
    headers = {"Authorization": f"Bearer {token}"}

    # build OData query
    qurl = build_odata_query(DATA_COLLECTION, FT_WKT, start_dt, end_dt, top=1000)
    print("OData query URL (truncated):", qurl[:300])

    r = requests.get(qurl, headers=headers, timeout=60)
    if r.status_code != 200:
        print("OData query failed:", r.status_code, r.text[:400])
        return

    json_ = r.json()
    values = json_.get("value", [])
    print(f"Found {len(values)} products in query response (value list).")

    if len(values) == 0:
        print("No products found for the chosen window/ROI.")
        return

    df = pd.DataFrame.from_dict(values)
    if "GeoFootprint" in df.columns:
        df["geometry"] = df["GeoFootprint"].apply(shape)
        gdf = gpd.GeoDataFrame(df).set_geometry("geometry")
    else:
        gdf = gpd.GeoDataFrame(df)

    if "Name" in gdf.columns:
        gdf = gdf[~gdf["Name"].str.contains("L1C", na=False)]

    print("Products after L1C filter:", len(gdf))

    session = requests.Session()
    session.headers.update({"Authorization": f"Bearer {token}"})

    for idx, row in gdf.iterrows():
        try:
            product_id = str(row.get("Id") or row.get("ID") or row.get("Identifier") or row.get("ProductId"))
            if not product_id:
                print(f"Skipping row {idx} — no Id field found: available keys: {list(row.index)}")
                continue

            identifier = None
            if "Name" in row and isinstance(row["Name"], str):
                identifier = row["Name"].split(".")[0]
            elif "identifier" in row:
                identifier = row["identifier"]
            else:
                identifier = product_id

            outname = f"{identifier}.zip"
            if os.path.exists(outname):
                print(f"{outname} already exists, skipping.")
                continue

            print(f"\nDownloading product {product_id} -> {outname}")
            try:
                token = get_keycloak_token(COPERNICUS_USER, COPERNICUS_PASSWORD)
                session.headers.update({"Authorization": f"Bearer {token}"})
                try_download_product(session, headers, product_id, outname)
            except Exception as e:
                print("Download failed, trying token refresh and retry...", e)
                time.sleep(1)
                token = get_keycloak_token(COPERNICUS_USER, COPERNICUS_PASSWORD)
                headers = {"Authorization": f"Bearer {token}"}
                session.headers.update(headers)
                try_download_product(session, headers, product_id, outname)
        except Exception as e:
            print(f"Failed to download product at index {idx}: {e}", file=sys.stderr)

    print("All done.")

if __name__ == "__main__":
    main()